In [1]:
import os
import json
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt


In [5]:
# Fonction pour charger les données JSON
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

# Charger les fichiers JSON
condition = load_json('/users/eleves-a/2022/janis.aiad/3A/EAP1/HFT_QR_RL/HFT_QR_RL/data/smash2/data/dbn/condition.json')
manifest = load_json('/users/eleves-a/2022/janis.aiad/3A/EAP1/HFT_QR_RL/HFT_QR_RL/data/smash2/data/dbn/manifest.json')
metadata = load_json('/users/eleves-a/2022/janis.aiad/3A/EAP1/HFT_QR_RL/HFT_QR_RL/data/smash2/data/csv/metadata.json')

# Fonction pour charger les données CSV
def load_csv(stock, date):
    file_path = f'/users/eleves-a/2022/janis.aiad/3A/EAP1/HFT_QR_RL/HFT_QR_RL/data/smash2/data/csv/{stock}/{date}.csv'
    return pd.read_csv(file_path)

# Spécifier les dates et les stocks
dates = ["20240624", "20240625", "20240626", "20240627", "20240628", "20240701", "20240702", "20240703", "20240705", "20240708", "20240709", "20240710", "20240711", "20240712", "20240715", "20240716", "20240717", "20240718", "20240719", "20240722", "20240723", "20240724", "20240725", "20240726", "20240729", "20240730", "20240731", "20240801", "20240802", "20240805", "20240806", "20240807", "20240808"]
stocks = ["HL","CGAU","RIOT","ASAI"]
# Charger les données pour chaque stock et chaque date dans des datasets différents
data_dict = {}
for stock in stocks:
    data_dict[stock] = {}
    for date in dates:
        data_dict[stock][date] = load_csv(stock, date).sample(frac=0.1, random_state=1)
# Concaténer toutes les données
data_list = [data_dict[stock][date] for stock in stocks for date in dates]
data = pd.concat(data_list, ignore_index=True)

# Filtrer par publisher_id = 39
data = data[data['publisher_id'] == 39]

# Convertir ts_event en datetime
data['ts_event'] = pd.to_datetime(data['ts_event'], utc=True)
data = data.sort_values(by='ts_event')


In [6]:
data.head()

,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,flags,...,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol
1013358,2024-06-24 13:08:09.783452293+00:00,10,39,1166,C,A,1,10.11,100,130,...,0,0,0,NaN,NaN,0,0,0,0,ASAI
1013528,2024-06-24 13:08:10.877605504+00:00,10,39,1166,A,B,0,9.95,100,130,...,0,0,0,NaN,NaN,0,0,0,0,ASAI
1011185,2024-06-24 13:08:10.877953925+00:00,10,39,1166,A,B,1,9.94,100,130,...,0,0,0,NaN,NaN,0,0,0,0,ASAI
1013859,2024-06-24 13:08:10.879240383+00:00,10,39,1166,A,A,0,10.07,100,130,...,0,0,0,NaN,NaN,0,0,0,0,ASAI
1013137,2024-06-24 13:08:12.877355668+00:00,10,39,1166,A,B,0,9.91,100,130,...,0,0,0,NaN,NaN,0,0,0,0,ASAI


In [7]:
print(data)

                                   ts_event  rtype  publisher_id  \
1013358 2024-06-24 13:08:09.783452293+00:00     10            39   
1013528 2024-06-24 13:08:10.877605504+00:00     10            39   
1011185 2024-06-24 13:08:10.877953925+00:00     10            39   
1013859 2024-06-24 13:08:10.879240383+00:00     10            39   
1013137 2024-06-24 13:08:12.877355668+00:00     10            39   
...                                     ...    ...           ...   
996199  2024-08-08 19:59:59.497580534+00:00     10            39   
1010426 2024-08-08 19:59:59.596968489+00:00     10            39   
1007452 2024-08-08 19:59:59.897619924+00:00     10            39   
1085682 2024-08-08 20:00:00.010915954+00:00     10            39   
1084691 2024-08-08 20:32:01.083716704+00:00     10            39   

         instrument_id action side  depth  price  size  flags  ...  ask_sz_08  \
1013358           1166      C    A      1  10.11   100    130  ...          0   
1013528           116

In [8]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

# Fonction pour calculer la corrélation avec la fonction identité
def calculate_correlation(data, bucket_number):
    data['imbalance_bucket'] = data['imbalance'].apply(lambda x: round(bucket_number * x) if pd.notna(x) else None)
    data = data.dropna(subset=['imbalance_bucket'])
    mean_delta_mid_price = data.groupby('imbalance_bucket')['delta_mid_price'].mean().reset_index()
    correlation, _ = pearsonr(mean_delta_mid_price['imbalance_bucket'], mean_delta_mid_price['delta_mid_price'])
    return correlation

# Fonction pour tracer et sauvegarder le graphique pour chaque stock
def plot_and_save(data, stock, bucket_number):
    data['imbalance_bucket'] = data['imbalance'].apply(lambda x: round(bucket_number * x) if pd.notna(x) else None)
    data = data.dropna(subset=['imbalance_bucket'])
    mean_delta_mid_price = data.groupby('imbalance_bucket')['delta_mid_price'].mean().reset_index()
    
    plt.figure(figsize=(14, 7))
    plt.plot(mean_delta_mid_price['imbalance_bucket'], mean_delta_mid_price['delta_mid_price'], marker='o', label='Mean Delta Mid Price')
    plt.title(f"Mean Delta Mid Price in Horizon 10 Trades vs Imbalance Buckets for {stock}")
    plt.xlabel(f"Imbalance Buckets (round({bucket_number}*imbalance))")
    plt.ylabel("Mean Delta Mid Price in Horizon 10 Trades")
    plt.legend()
    plt.savefig(f"imbalance_plot_{stock}.png")
    plt.close()

# Calculer l'imbalance des meilleures offres et demandes
for df in data_list:
    df['imbalance'] = (df['bid_sz_00'] - df['ask_sz_00']) / (df['bid_sz_00'] + df['ask_sz_00'])

# Calculer le prix moyen
for df in data_list:
    df['mid_price'] = (df['bid_px_00'] + df['ask_px_00']) / 2

# Calculer le delta du prix moyen sur un horizon de 10 transactions
for df in data_list:
    df['delta_mid_price'] = df['mid_price'].diff(periods=100)

# Filtrer les données pour ne conserver que les lignes où delta_mid_price n'est pas NaN
data_filtered_list = [df.dropna(subset=['delta_mid_price']) for df in data_list]

# Concaténer toutes les données filtrées
all_data = pd.concat(data_filtered_list, ignore_index=True)

# Liste pour stocker les meilleurs bucket_numbers
best_buckets = []

# Appliquer la fonction pour chaque stock
for stock in stocks:
    stock_data = all_data[all_data['symbol'] == stock]
    if not stock_data.empty:
        best_correlation = -1
        best_bucket_number = 5
        for bucket_number in range(5, 11):
            correlation = calculate_correlation(stock_data, bucket_number)
            if correlation > best_correlation:
                best_correlation = correlation
                best_bucket_number = bucket_number
        best_buckets.append((stock, best_bucket_number))
        plot_and_save(stock_data, stock, best_bucket_number)

# Afficher la liste des meilleurs bucket_numbers
print(best_buckets)


/tmp/ipykernel_2817338/1465815001.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['imbalance_bucket'] = data['imbalance'].apply(lambda x: round(bucket_number * x) if pd.notna(x) else None)
/tmp/ipykernel_2817338/1465815001.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['imbalance_bucket'] = data['imbalance'].apply(lambda x: round(bucket_number * x) if pd.notna(x) else None)
/tmp/ipykernel_2817338/1465815001.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

[('HL', 6), ('CGAU', 5), ('RIOT', 5), ('ASAI', 5)]
